<a href="https://colab.research.google.com/github/franklinspspsp/IA-Waifu/blob/main/editor_de_waifus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0 - Instalacion**

In [ ]:
#@title <- Haz click para instalar **-Hay mucha gente utilizandolo y ahora tarda mas (10 min). ¡Paciencia!**
#markdown 

%cd /content

#Apaño rapido demasiada gente
!pip install mega.py
from mega import Mega
mega = Mega()
m = mega.login()
from IPython.display import clear_output
clear_output()

#DESCARGA DE STYLEGAN, MODELOS Y DIRECIONES
import gdown
import os.path



!rm -rf sample_data
if not os.path.isdir('/content/stylegan2'):
  !git clone https://github.com/shawwn/stylegan2 -b estimator /content/stylegan2

#DESCARGA DIRECCIONES Y RED NEURONAL (TIPLICADO POR SI DEMASIADO GENTE LO UTILIZA)
if not os.path.isfile('/content/tadne-directions.zip'):
  gdown.download('https://drive.google.com/uc?id=1-YzKBgHtY6RAGJx8h2u3jhWb_qvXXwLL', 'tadne-directions.zip', quiet=False)
if not os.path.isfile('/content/tadne-directions.zip'):
  gdown.download('https://drive.google.com/uc?id=1h_ylMTq7lLiWAx_AObqlOXKpA1Nt-Vfi', 'tadne-directions.zip', quiet=False)
if not os.path.isfile('/content/tadne-directions.zip'):
  gdown.download('https://drive.google.com/uc?id=1dqRXpWhQ95AO0logLvTBMLVuIGCD7AXk', 'tadne-directions.zip', quiet=False)
if not os.path.isfile('/content/tadne-directions.zip'):
  m.download_url('https://mega.nz/file/lx4C1ZyA#hLVLYqq6ts76X63-AFySa771J6zwGCtpj6Oo-UJUmvA')

if not os.path.isfile('/content/network-tadne.pkl'): 
  gdown.download('https://drive.google.com/uc?id=120gB3-xloT2owT4q6LVVuPZOfuoornT7', 'network-tadne.pkl', quiet=False)
if not os.path.isfile('/content/network-tadne.pkl'): 
  gdown.download('https://drive.google.com/uc?id=15mrjZbCFTOJKPW-6ONvH5g72HIAxvIXF', 'network-tadne.pkl', quiet=False)
if not os.path.isfile('/content/network-tadne.pkl'):
  gdown.download('https://drive.google.com/uc?id=1A-E_E32WAtTHRlOzjhhYhyyBDXLJN9_H', 'network-tadne.pkl', quiet=False)
if not os.path.isfile('/content/network-tadne.pkl'):
  clear_output()
  print('¡Hay mucha gente utilizandolo! Va a tardar un poco mas en instalarse... ¡Paciencia!')
  m.download_url('https://mega.nz/file/V9gijLIL#OUZC-nOVs8zhKjnGtq_15EvgXnqsCCUreA8NU3kTNcs')
  


if not os.path.isfile('/content/deepdanbooru_dirs.pkl'):
  gdown.download('https://drive.google.com/uc?id=1uVUCSRS0tPBKgOa0fvq9U85u7uG8wPPP', 'deepdanbooru_dirs.pkl', quiet=False)
if not os.path.isfile('/content/deepdanbooru_dirs.pkl'):
  gdown.download('https://drive.google.com/uc?id=1rHH1ivpCBsRfchdz72r81Qqu0WnJPcqN', 'deepdanbooru_dirs.pkl', quiet=False)
if not os.path.isfile('/content/deepdanbooru_dirs.pkl'):
  clear_output()
  print('Ya falta poco!')
  m.download_url('https://mega.nz/file/x1ZzwITA#nemK8eRuPUWa-eOiw37gWEVYya2HZjgZrz5ZuLTxDA4')
if not os.path.isfile('/content/deepdanbooru_dirs.pkl'):
  gdown.download('https://drive.google.com/uc?id=1x9cD_MV3UVboqnlpq9ZPWQ-kvFzxOGaz', 'deepdanbooru_dirs.pkl', quiet=False)




if not os.path.isdir('directions/'):
  !mkdir directions
  !unzip -q /content/tadne-directions.zip -d directions
  !rm /content/directions/StyleGAN2-Light_direction-ffhq-ipca-w-style-comp15-range8-9.pkl


%tensorflow_version 1.x
%cd /content/stylegan2

import os
import pickle
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import scipy
import tensorflow as tf
import tflex
from tqdm import tqdm

if 'COLAB_TPU_ADDR' in os.environ:
    os.environ['TPU_NAME'] = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    os.environ['NOISY'] = '1'


#CARGA MODELO
print('\nCargando el modelo...')
tflib.init_tf()
sess = tf.get_default_session()
sess.list_devices()
cores = tflex.get_cores()
tflex.set_override_cores(cores)
print('Paciencia...')
try:
  _G, _D, Gs = pickle.load(open("/content/network-tadne.pkl", "rb"))

  #CARGA DIRECCIONES
  print('Cargando direcciones...')
  deepdanbooru_dirs = pickle.load(open('/content/deepdanbooru_dirs.pkl', 'rb'))

  named_directions = {}
  latent_dirs = []
  comp = 0
except:
  import IPython 
  from IPython.display import clear_output
  clear_output()
  display(IPython.display.HTML(
      '''
      <marquee scrollamount="15" style='width: 80%; font-size: 3em ; color: red' scrollamount="20"><b>HAY DEMASIADAS PERSONAS UTILIZANDOLO, POR FAVOR VUELVE MAS TARDE</b></marquee>
      <h1>Hay demasiadas personas utilizando el enlace y Google lo ha bloqueado ¡Vuelve a intentarlo en unos minutos! :)</h1>
      <h2>Con una buena web esto no pasaría... jajajaj</h2>
      <h3>Puedes ver un vídeo en lo que esperas :)</h3>
      '''
  ))

for tag, vec in tqdm(deepdanbooru_dirs.items()):
    for layers in [(0, 6), (6, 12), (12, 16)]:
        name = tag + "-" + str(layers[0]) + "-" + str(layers[1])
        named_directions[f'{name}'] = [comp, int(layers[0]), int(layers[1]), f'{name}']
    latent_dirs.append(np.matrix(vec[0]))
    comp += 1

#RESTO DE FUNCIONES
import PIL
from PIL import Image
import numpy as np
import IPython
from IPython.display import clear_output

final_image = None
def generate_image_from_w(w, truncation_psi):
    with tflex.device('/gpu:0'):
        Gs_kwargs = dnnlib.EasyDict()
        Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
        Gs_kwargs.randomize_noise = False
        if truncation_psi is not None:
            Gs_kwargs.truncation_psi = truncation_psi
        synthesis_kwargs = dict(output_transform=Gs_kwargs.output_transform, truncation_psi=truncation_psi, minibatch_size=1)
        images = Gs.components.synthesis.run(w, randomize_noise=False, **synthesis_kwargs)
        global final_image
        final_image = PIL.Image.fromarray(images[0], 'RGB')
        display(final_image.resize((500,500), Image.LANCZOS))

def normalize(v):
    norm=np.linalg.norm(v, ord=2)
    if norm==0:
        norm=np.finfo(v.dtype).eps
    return v/norm * len(v)
    
loaded_w = None
def display_sample(seed, truncation, direction, distance, scale, start, end, update, disp=True, save=None, noise_spec=None, **args):
    global loaded_w
    if update == False:
        print("False")
    rng = np.random.RandomState(seed)
    z = rng.standard_normal(*Gs.input_shape[1:]).reshape(1, *Gs.input_shape[1:])
    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.randomize_noise = False

    all_w = Gs.components.mapping.run(z, None, **Gs_kwargs) # [minibatch, layer, component]
    if loaded_w is not None:
      all_w = loaded_w.copy()
    
    for i, item in enumerate(args):
        value = args[item]
        start_l = named_directions[item][1]
        end_l = min(16, named_directions[item][2])
        direction_l = normalize(latent_dirs[named_directions[item][0]])
        for l in range(start_l, end_l):
            all_w[0][l] = all_w[0][l] + direction_l * value * scale

    if truncation != 1:
        w_avg = Gs.get_var('dlatent_avg')
        all_w = w_avg + (all_w - w_avg) * truncation # [minibatch, layer, component]

    generate_image_from_w(all_w, truncation)


#PREPARAR DISPLAY
import ipywidgets as widgets
from ipywidgets import fixed
from ipywidgets import Layout, Button, Box, VBox, Label
from IPython.display import clear_output 
from google.colab import files


seed = np.random.randint(0,100000)

style = {'description_width': 'initial', 'width': '300px'}
seed = widgets.IntSlider(min=0, max=100000, step=1, value=seed, style=style, description='Seed: ',   continuous_update=False)
truncation = widgets.FloatSlider(min=0, max=2, step=0.1, value=0.7, style=style, description='Truncation: ', continuous_update=False)
ui = widgets.HBox([seed, truncation])
ui.layout.width = '600px'

style = {'description_width': 'initial', 'width': '100px'}
distance = widgets.FloatSlider(min=-10, max=10, step=0.1, value=0, description='Distance: ', continuous_update=False, style=style)
scale = widgets.FloatSlider(min=0, max=10, step=0.05, value=1, description='Scale: ', continuous_update=False)
start_layer = widgets.IntSlider(min=0, max=18, step=1, value=18, description='start layer: ', continuous_update=False)
end_layer = widgets.IntSlider(min=0, max=18, step=1, value=18, description='end layer: ', continuous_update=False)

update = widgets.Checkbox(value=True, description="update")

directions_list = []
params = {'seed': seed, 'truncation': truncation, 'direction': fixed(0), 'distance': distance, 'scale': scale, 'start': start_layer, 'end': end_layer, 'update' : update}

for i, item in enumerate(named_directions):
    name = named_directions[item][3]
    widget = widgets.FloatSlider(min=-20, max=20, step=0.1, value=0, description=name + ': ', continuous_update=False, style=style, layout={'width' : 'auto'})
    directions_list.append(widget)
    params[item] = widget

grid = widgets.GridspecLayout(len(directions_list)//3, 3, width='90%', height="300px")
for i in range(len(directions_list)//3):
    for j in range(3):
        grid[i, j] = directions_list[3*i+j]
ui2 = grid


reset = widgets.Button(description="Resetea sliders")
save_btn = widgets.Button(description="Guardar")
download_img_btn = widgets.Button(description="Descargar Imagen")
download_npy_btn = widgets.Button(description="Descargar .npy")

def reset_sliders(b):
    directions_list = []
    params_new = {'seed': seed, 'truncation': truncation, 'direction': fixed(0), 'distance': distance, 'scale': scale, 'start': start_layer, 'end': end_layer, 'update' : update}
    for i, item in enumerate(named_directions):
        name = named_directions[item][3]
        widget = widgets.FloatSlider(min=-20, max=20, step=0.1, value=0, description=name + ': ', continuous_update=False, style=style, layout={'width' : 'auto'})
        directions_list.append(widget)
        params_new[item] = widget
    params = params_new
    grid = widgets.GridspecLayout(len(directions_list)//3, 3, width='90%', height="300px")
    for i in range(len(directions_list)//3):
        for j in range(3):
            grid[i, j] = directions_list[3*i+j]
        
    ui2 = grid
    clear_output()
    out = widgets.interactive_output(display_sample, params)

    display(ui, out, ui2, reset, save_btn, download_img_btn, download_npy_btn)


def save_w(b):
    rng = np.random.RandomState(params['seed'].value)
    z = rng.standard_normal(*Gs.input_shape[1:]).reshape(1, *Gs.input_shape[1:])

    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.randomize_noise = False

    all_w = Gs.components.mapping.run(z, None, **Gs_kwargs) # [minibatch, layer, component]

    for i, item in enumerate(params):
        if item not in ['seed', 'truncation', 'direction', 'distance', 'scale', 'start', 'end', 'update']:
            value = params[item].value
            start_l = named_directions[item][1]
            end_l = min(16, named_directions[item][2])
            direction_l = normalize(latent_dirs[named_directions[item][0]])
            for l in range(start_l, end_l):
                all_w[0][l] = all_w[0][l] + direction_l * value * params['scale'].value

    if truncation != 1:
        w_avg = Gs.get_var('dlatent_avg')
        all_w = w_avg + (all_w - w_avg) * params['truncation'].value # [minibatch, layer, component]
    
    if not os.path.isdir('/content/stylegan2/saves/'):
      os.mkdir('/content/stylegan2/saves')

    files_number = len(os.listdir('/content/stylegan2/saves/'))
    out_name = 'image' + str(files_number +1).zfill(3)
    np.save('/content/stylegan2/saves/'+ out_name + ".npy", all_w)
    print("Guardado: ", out_name + ".npy")
  

def download_npy(b):
    rng = np.random.RandomState(params['seed'].value)
    z = rng.standard_normal(*Gs.input_shape[1:]).reshape(1, *Gs.input_shape[1:])

    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.randomize_noise = False

    all_w = Gs.components.mapping.run(z, None, **Gs_kwargs) # [minibatch, layer, component]

    for i, item in enumerate(params):
        if item not in ['seed', 'truncation', 'direction', 'distance', 'scale', 'start', 'end', 'update']:
            value = params[item].value
            start_l = named_directions[item][1]
            end_l = min(16, named_directions[item][2])
            direction_l = normalize(latent_dirs[named_directions[item][0]])
            for l in range(start_l, end_l):
                all_w[0][l] = all_w[0][l] + direction_l * value * params['scale'].value

    if truncation != 1:
        w_avg = Gs.get_var('dlatent_avg')
        all_w = w_avg + (all_w - w_avg) * params['truncation'].value # [minibatch, layer, component]
    
    np.save("imagen.npy", all_w)
    files.download("imagen.npy")

def download_img(b):
  global final_image
  final_image.save("descarga.jpg", "JPEG")
  files.download("descarga.jpg")

def load_w(b):
    global loaded_w
    loaded_w = np.load("test.npy")

reset.on_click(reset_sliders)
save_btn.on_click(save_w)
download_img_btn.on_click(download_img)
download_npy_btn.on_click(download_npy)
out = widgets.interactive_output(display_sample, params)


clear_output()

#HTML MENSAJE DE RESULTADO
if latent_dirs and Gs:
  display(IPython.display.HTML(
      '''
      <marquee scrollamount="15" style='width: 80%; font-size: 3em ; color: green;'><b>¡Instalación exitosa!</b></marquee>
      <div style = "display: flex; justify-content: center; align-items: center;">
        <img src="https://media.8kun.top/t/src/1444067143926.gif" alt="Snow" style="height:30%;">
        <div style="position: absolute; color:green; font-size:10vh  "></div>
      </div>
      '''
  ))
  #https://i.pinimg.com/originals/38/1b/99/381b99f3cc90265f4a9499ee8008dd99.gif
  #https://media.8kun.top/t/src/1444067143926.gif
else:
  display(IPython.display.HTML(
      '''
      <marquee scrollamount="15" style='width: 80%; font-size: 3em ; color: red' scrollamount="20"><b>¡ERROR AL INSTALAR!</b></marquee>
      <h1>Posibles soluciones:</h1>
      <ol> Vuelve a intentar la instalación (paso 0).
        <li>Vuelve a intentarlo</li>
        <li>Si sigue apareciendo un error, ve al menú > Entorno de ejecución > Restablecer estado de fábrica del entorno de ejecución y empieza todo otra vez.</li>
        <li>Es posible que haya mucha gente usándolo y se haya caido. Espera un rato y ¡vuelve a intentarlo!. Puedes ver un video de <a href="https://www.youtube.com/c/brunvelop">mi canal</a> en lo que esperas 😄</li>
        <li>Si después de todo esto sigues teniendo algún error dimelo en cualquier red social e intentaré ayudarte 😄</li>
      </ol> 
      '''
  ))

from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/9/91/Sound4.wav").play()')

# **1 - Waifu 3000**

In [ ]:
#@title <- Carga la UI (~ 2 min) - ¡Espera hasta que se vean todos los sliders!
display(ui, out, ui2, reset, save_btn, download_img_btn, download_npy_btn)

from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/9/91/Sound4.wav").play()')

# **2 - Genera vídeo**

In [ ]:
#@title <- Visualiza las imagenes entre las que se generará el vídeo
def preview_files():
    for file in sorted(os.listdir('/content/stylegan2/saves'), reverse = False):
      truncation = 1
      w = np.load('/content/stylegan2/saves/'+file)
      noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
      Gs_kwargs = dnnlib.EasyDict()
      Gs_kwargs.randomize_noise = False

      if truncation != 1:
          w_avg = Gs.get_var('dlatent_avg')
          all_w = w_avg + (all_w - w_avg) * truncation # [minibatch, layer, component]

      generate_image_from_w(w, truncation)

preview_files()


In [ ]:
#@title Crea el vídeo (~ 1 min por segundo de duración)
duracion = 14 #@param {type:"slider", min:1, max:60, step:0.1}
fps = 23 #@param {type:"slider", min:10, max:120, step:1}
loop = True #@param {type:"boolean"}

from IPython.display import display, HTML
from IPython.display import HTML
import math
from PIL import ImageFont
from PIL import ImageDraw
import os
import moviepy.editor
from datetime import datetime
import moviepy.editor as mpy
from IPython.display import clear_output 

def interpolate_between_ws(seed_array, truncation=0.5, duration_sec = 12.0, smoothing_sec = 0, mp4_fps = 20, filename=None, text=False):
    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_kwargs.randomize_noise = False
    synthesis_kwargs = dict(output_transform=Gs_kwargs.output_transform, truncation_psi=truncation, minibatch_size=8)
    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
    
    all_w = seed_array
        
    num_frames = int(np.rint(duration_sec * mp4_fps))
        
    def make_frame(t):
        blend = ((len(seed_array)-1)*t/duration_sec)%1.0
        src_i = math.floor((t/duration_sec)*(len(seed_array)-1))
        dst_i = src_i + 1
        #print(t, blend, src_i, dst_i)
        all_w_new = (blend * all_w[dst_i]) + (1 - blend) * all_w[src_i]
        all_images_src = Gs.components.synthesis.run(all_w_new, randomize_noise=False, **synthesis_kwargs)
        #all_images_dst = Gs.components.synthesis.run(all_w_dst, randomize_noise=False, **synthesis_kwargs)
        if text:
            new_im = PIL.Image.new('RGB', (512, 600))
            new_im.paste(PIL.Image.fromarray(np.median(all_images_src, axis=0).astype(np.uint8)), (0, 0))
            draw = ImageDraw.Draw(new_im)
            font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf", size=16)
            draw.text((10, 512), "{:0.2f}".format((1-blend)), (255, 0, 0), font=font)
            draw.text((50, 512), str(seed_array[src_i]), (255, 255, 255), font=font)
            draw.text((10, 550), "{:0.2f}".format((blend)), (0, 255, 0), font=font)
            draw.text((50, 550), str(seed_array[dst_i]), (255, 255, 255), font=font)
            return np.array(new_im)
        else:
            return all_images_src[0]

    np.save("vector.npy", all_w)
    mp4_file = 'video.mp4'
    mp4_codec = 'libx264'
    mp4_bitrate = '5M'

    video_clip = moviepy.editor.VideoClip(make_frame, duration=duration_sec)
    video_clip.write_videofile(mp4_file, fps=mp4_fps, codec=mp4_codec, bitrate=mp4_bitrate)
    
    return mp4_file


w_arr = []
files = sorted(os.listdir('/content/stylegan2/saves'), reverse = True)
for file in files:
  w_val = np.load('/content/stylegan2/saves/'+file)
  w_arr.append(w_val)

if loop:
  w_val = np.load('/content/stylegan2/saves/'+files[0])
  w_arr.append(w_val)

output_file = interpolate_between_ws(w_arr, duration_sec = duracion, mp4_fps = fps)
clear_output()
display(mpy.ipython_display("video.mp4", autoplay=1, loop=1))

# **3 - Borra tus imágenes guardadas**

In [ ]:
#@title ¡CUIDADO! Tus imágenes se perderán pero podrás empezar de nuevo
!rm -rf saves/*